In [1]:
url  =""
project = "milo" 

In [30]:
def apply_threshold(probabilities, threshold=0.28):
    return (probabilities >= threshold).astype(int)

In [36]:
def map_predictions(predictions, labels):
    # Map each binary prediction to its corresponding label or 'not_LA'
    mapped_predictions = []
    for prediction in predictions:
        # Get the labels for which the prediction is 1
        label_predictions = [label for pred, label in zip(prediction, labels) if pred == 1]
        
        # Join the labels into a string, or return 'not_LA' if no labels are present
        prediction_str = ','.join(label_predictions) if label_predictions else 'not_LA'
        mapped_predictions.append(prediction_str)
    
    return mapped_predictions

In [41]:
!python app.py

 * Serving Flask app 'app'
 * Debug mode: on
 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat
 * Debugger is active!
 * Debugger PIN: 700-184-686
127.0.0.1 - - [06/Apr/2024 12:48:32] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [06/Apr/2024 12:48:33] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [06/Apr/2024 12:48:44] "POST /submit HTTP/1.1" 500 -
Traceback (most recent call last):
  File "/Users/devang/miniconda3/envs/flask/lib/python3.11/site-packages/flask/app.py", line 1488, in __call__
    return self.wsgi_app(environ, start_response)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/devang/miniconda3/envs/flask/lib/python3.11/site-packages/flask/app.py", line 1466, in wsgi_app
    response = self.handle_exception(e)
               ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/devang/miniconda3/envs/flask/lib/python3.11/site-packages/flask/app.py", line 1463, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^

In [40]:
import repodownloader 
import ktrain
import pandas as pd
from ktrain import tabular
import numpy as np
from sklearn.utils import shuffle
from sklearn.metrics import matthews_corrcoef, roc_auc_score
from sklearn.metrics import roc_curve, auc
import os
import matplotlib.pyplot as plt
def eval(url,project):
    if os.path.exists(f"{project}_data.csv"):
        print("File already exists")
    else:
        repodownloader.extract_java_methods_from_project(url,project)
    file = f"{project}_data.csv"
    model_path = f"model/buildship_final_model"
    df = pd.read_csv(file)
    df = df.drop(columns=['comments'])
    df['index']=df.index
    copy_df = df.copy()
    df = df.drop(columns=['path'])
    df["is"] = 0
    df["set"] = 0
    df["get"] = 0
    df["DOS"] = 0
    predictor = ktrain.load_predictor(model_path)
    pred = predictor.predict(df, return_proba=True)
    pred = apply_threshold(pred)
    print(pred[0])
    labels = ['is', 'set', 'get', 'DOS']
    mapped_predictions = map_predictions(pred, labels)
    copy_df['LABEL'] = mapped_predictions
    pred = predictor.predict(df)
    copy_df['PREDICTION'] = pred
    return copy_df

In [38]:
eval_df,pred = eval(url,project)

File already exists
['is', 'set', 'get', 'DOS']
   is  set  get  DOS
0   0    0    0    0
1   0    0    0    0
2   0    0    0    0
3   0    0    0    0
4   0    0    0    0
[0 0 0 0]


In [39]:
eval_df.to_csv(f"{project}_eval.csv",index=False)

In [ ]:
type(pred[0])

numpy.float32

In [15]:
print(eval_df.head())

  access_modifier                    return_type          method_name  \
0          public                 ResponseHeader    getResponseHeader   
1          public  Class<RegisterServerResponse>              getType   
2          public         RegisterServerResponse               decode   
3          public                 ExpandedNodeId            getTypeId   
4          public                 ExpandedNodeId  getBinaryEncodingId   

                                        parameters  \
0                                              NaN   
1                                              NaN   
2  SerializationContext context, UaDecoder decoder   
3                                              NaN   
4                                              NaN   

                                             returns  \
0                            return responseHeader;    
1              return RegisterServerResponse.class;    
2  return new RegisterServerResponse(responseHead...   
3           